## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.


STEPS TO FOLLOW

1) Understand the data

2) Quick Notes

3) Action plan

4) Plan Execution with sample
    
5) Plan Execution with all data


# 1) Lets check what we have

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  

In [3]:
bookings_check = pd.read_csv('bookings.csv', sep= '^', error_bad_lines=False, nrows=1)
pd.set_option("display.max_columns", None)
bookings_check

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL


In [4]:
searches_check = pd.read_csv('searches.csv', sep= '^', error_bad_lines=False, nrows=1)
searches_check

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA


# 2) Quick Notes:

Before taking an action plan I want to check:

    1) Q: Difference between act_date and cre_date (bookings). 
       A: Both are always before brd_time. It seems cre_date is when the account was created or system detected the user for the first time and act_date is when the flight was booked in the system. The reason to make this assumption is that cre_data is always less than act_data and all act_date is from 2013, while some of cre_dates are dated in 2011 or 2012.
       
    2) Q: Have both datasets all the months of the year 2013? 
       A: Both datasets have the 12 months of 2013
       
       
    

# 3) Action Plan:

Information that appears in both datasets:
- Search day and booking day: bookings['act_date'], searches ['Date']
- Origin and destination: bookings[['dept_port', 'arr_port'], searches[['Origin', 'Destination']]
- Boarding time: bookings['brd_time'], searches['Seg1Date']
- Pax: bookings['pax'] has to be > 1 as cancelations are not taken into account. People dont cancel through searches.




#### 1. Discard all no needed information:

- Drop duplicates

#### 2. Create both df with the 4 columns

#### 3. Change data to correct type and deal with NaN
- Change dates to datetime
- Find out how many NaN and evaluate if it makes sense TO DO something OR not
- Delete spaces for string in iata codes of bookings
- Create rank

#### 4. Merge data

- Check if size is the same and data make sense
- Filter bookings by rank 1 and searches by rank 1 to have the ones that are not repeated
- Add the 1 and 0 to the raw search df
     

# 4) Plan Execution with Sample:

In [5]:
#Sample of 2M rows
bookings_raw = pd.read_csv('bookings.csv', sep= '^', error_bad_lines=False, nrows=2000000)
searches_raw = pd.read_csv('searches.csv', sep= '^', error_bad_lines=False, nrows=2000000)

C:\Users\ORTEGA\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (40,41,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#Drop duplicates
bookings_nodup = bookings_raw.drop_duplicates()
searches_nodup = searches_raw.drop_duplicates()

In [7]:
# Reseting index to follow an order from 0 to nrows
bookings_nodup.reset_index(inplace=True)
bookings_nodup.drop('index', axis = 1, inplace=True)

searches_nodup.reset_index(inplace=True)
searches_nodup.drop('index', axis = 1, inplace=True)

In [8]:
#Selecting useful columns
bookings_cols_nodup = bookings_nodup[['act_date           ','dep_port', 'arr_port','brd_time           ','pax']]
searches_cols_nodup = searches_nodup[['Date','Origin','Destination','Seg1Date']]

#bookings with pax>1
bookings_cols_nodup = bookings_cols_nodup[bookings_cols_nodup['pax'] > 0]

In [9]:
#Drop NaN
bookings_cols_nodup_nonan = bookings_cols_nodup.dropna()
searches_cols_nodup_nonan = searches_cols_nodup.dropna()

In [10]:
print('bookings_raw shape = ', bookings_raw.shape,'\nbookings without duplicates = ', bookings_nodup.shape, '\nbookings selected columns and no duplicates =', bookings_cols_nodup.shape,'\nbookings selected columns, no duplicates and no Nan=', bookings_cols_nodup_nonan.shape)
print("")
print('searches_raw shape = ', searches_raw.shape,'\nsearches without duplicates = ', searches_nodup.shape, '\nsearches selected columns and no duplicates =', searches_cols_nodup.shape,'\nsearches selected columns, no duplicates and no Nan=', searches_cols_nodup_nonan.shape)  

bookings_raw shape =  (2000000, 38) 
bookings without duplicates =  (1000000, 38) 
bookings selected columns and no duplicates = (680866, 5) 
bookings selected columns, no duplicates and no Nan= (680866, 5)

searches_raw shape =  (2000000, 45) 
searches without duplicates =  (388369, 45) 
searches selected columns and no duplicates = (388369, 4) 
searches selected columns, no duplicates and no Nan= (387276, 4)


In [11]:
# Changing dates to datetime to have same format in both df and delete the hour
import datetime as dt
#First for the creation date of the booking and search
bookings_cols_nodup_nonan['created_date'] = pd.to_datetime(bookings_cols_nodup_nonan['act_date           '], errors='coerce', format='%Y-%m-%d').dt.date
searches_cols_nodup_nonan['created_date'] = pd.to_datetime(searches_cols_nodup_nonan['Date'], errors='coerce', format='%Y-%m-%d').dt.date

#Second for the boarding time
bookings_cols_nodup_nonan['board_date'] = pd.to_datetime(bookings_cols_nodup_nonan['brd_time           '], errors='coerce', format='%Y-%m-%d').dt.date
searches_cols_nodup_nonan['board_date'] = pd.to_datetime(searches_cols_nodup_nonan['Seg1Date'], errors='coerce', format='%Y-%m-%d').dt.date


In [12]:
#Drop old columns with dates and pax as we don't need them anymore
bookings_datetime = bookings_cols_nodup_nonan.drop(['act_date           ','brd_time           '], axis =1)
searches_datetime = searches_cols_nodup_nonan.drop(['Date', 'Seg1Date'], axis = 1)

In [13]:
#Changing column names of bookings to match searches to make the merge easier
bookings_datetime['Origin'] = bookings_datetime['dep_port'].str.split(" ", n = 1, expand = True)[0]
bookings_datetime['Destination'] = bookings_datetime['arr_port'].str.split(" ", n = 1, expand = True)[0]

#Droping no needed columns: pax as it is already filtered and the dep_port arr_port
bookings_datetime.drop(['pax','dep_port', 'arr_port'], axis = 1, inplace = True)



In [14]:
#Changing column names of searches to make sure
searches_datetime['Origin'] = searches_datetime['Origin'].str.split(" ", n = 1, expand = True)[0]
searches_datetime['Destination'] = searches_datetime['Destination'].str.split(" ", n = 1, expand = True)[0]


# RANK

In [109]:
searches_datetime

,Origin,Destination,created_date,board_date
0,TXL,AUH,2013-01-01,2013-01-26
1,ATH,MIL,2013-01-01,2013-01-04
2,ICT,SFO,2013-01-01,2013-08-02
3,RNB,ARN,2013-01-01,2013-01-02
4,OSL,MAD,2013-01-01,2013-03-22
...,...,...,...,...
388364,HLZ,BNE,2013-01-08,2013-10-26
388365,PEK,SEL,2013-01-08,2013-03-22
388366,YUL,NCL,2013-01-08,2013-01-27
388367,MIA,NCE,2013-01-08,2013-01-23


In [110]:
searches_rank = searches_datetime.copy()

In [112]:
#Converting board date to str as later am going to zip it and create a string from a tuple
searches_rank['board_date'] = searches_rank['board_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [114]:
#Zipping columns to obtaing an Id name with Origin, Destination and board date 
searches_rank['zip'] = tuple(zip(searches_rank.Origin, searches_rank.Destination, searches_rank.board_date))

In [115]:
searches_rank

,Origin,Destination,created_date,board_date,zip
0,TXL,AUH,2013-01-01,2013-01-26,"(TXL, AUH, 2013-01-26)"
1,ATH,MIL,2013-01-01,2013-01-04,"(ATH, MIL, 2013-01-04)"
2,ICT,SFO,2013-01-01,2013-08-02,"(ICT, SFO, 2013-08-02)"
3,RNB,ARN,2013-01-01,2013-01-02,"(RNB, ARN, 2013-01-02)"
4,OSL,MAD,2013-01-01,2013-03-22,"(OSL, MAD, 2013-03-22)"
...,...,...,...,...,...
388364,HLZ,BNE,2013-01-08,2013-10-26,"(HLZ, BNE, 2013-10-26)"
388365,PEK,SEL,2013-01-08,2013-03-22,"(PEK, SEL, 2013-03-22)"
388366,YUL,NCL,2013-01-08,2013-01-27,"(YUL, NCL, 2013-01-27)"
388367,MIA,NCE,2013-01-08,2013-01-23,"(MIA, NCE, 2013-01-23)"


In [117]:
#Droping columns we dont need anymore
search_created_zip = searches_rank.drop(['Origin', 'Destination', 'board_date'], axis = 1)

In [118]:
search_created_zip

,created_date,zip
0,2013-01-01,"(TXL, AUH, 2013-01-26)"
1,2013-01-01,"(ATH, MIL, 2013-01-04)"
2,2013-01-01,"(ICT, SFO, 2013-08-02)"
3,2013-01-01,"(RNB, ARN, 2013-01-02)"
4,2013-01-01,"(OSL, MAD, 2013-03-22)"
...,...,...
388364,2013-01-08,"(HLZ, BNE, 2013-10-26)"
388365,2013-01-08,"(PEK, SEL, 2013-03-22)"
388366,2013-01-08,"(YUL, NCL, 2013-01-27)"
388367,2013-01-08,"(MIA, NCE, 2013-01-23)"


In [120]:
#Converting tuple to string 
search_created_zip['zip'] = search_created_zip['zip'].apply(lambda x: ''.join(x))

In [121]:
search_created_zip

,created_date,zip
0,2013-01-01,TXLAUH2013-01-26
1,2013-01-01,ATHMIL2013-01-04
2,2013-01-01,ICTSFO2013-08-02
3,2013-01-01,RNBARN2013-01-02
4,2013-01-01,OSLMAD2013-03-22
...,...,...
388364,2013-01-08,HLZBNE2013-10-26
388365,2013-01-08,PEKSEL2013-03-22
388366,2013-01-08,YULNCL2013-01-27
388367,2013-01-08,MIANCE2013-01-23


In [123]:
#I make sure created-date its in datetime format as its going to be needed later for the groupby
search_created_zip['created_date'] =  pd.to_datetime(search_created_zip['created_date'])

In [125]:
# I cant groupby by created_date and zip without a value to groupby with.
# Becouse of this a fake columns is created with a value of 1 to be able to groupby by created date and zip
search_created_zip['fake_columns'] = 1

In [126]:
# Hago groupoby por created date y zip
search_created_zip['rank_search'] = search_created_zip.groupby(['created_date', 'zip']).rank(method= 'first')

In [127]:
search_created_zip

,created_date,zip,fake_columns,rank_search
0,2013-01-01,TXLAUH2013-01-26,1,1.0
1,2013-01-01,ATHMIL2013-01-04,1,1.0
2,2013-01-01,ICTSFO2013-08-02,1,1.0
3,2013-01-01,RNBARN2013-01-02,1,1.0
4,2013-01-01,OSLMAD2013-03-22,1,1.0
...,...,...,...,...
388364,2013-01-08,HLZBNE2013-10-26,1,2.0
388365,2013-01-08,PEKSEL2013-03-22,1,2.0
388366,2013-01-08,YULNCL2013-01-27,1,2.0
388367,2013-01-08,MIANCE2013-01-23,1,2.0


In [129]:
# As we can observe in de df below, when created date and name match, rank increase in + 1 for repeated values (created date)
search_created_zip[search_created_zip['zip'] == 'NCELGW2014-06-28']

,created_date,zip,fake_columns,rank_search
196188,2013-07-16,NCELGW2014-06-28,1,1.0
201365,2013-07-21,NCELGW2014-06-28,1,1.0
206462,2013-07-26,NCELGW2014-06-28,1,1.0
210046,2013-07-30,NCELGW2014-06-28,1,1.0
230505,2013-08-19,NCELGW2014-06-28,1,1.0
251167,2013-09-09,NCELGW2014-06-28,1,1.0
276998,2013-10-04,NCELGW2014-06-28,1,1.0
277619,2013-10-05,NCELGW2014-06-28,1,1.0
279480,2013-10-07,NCELGW2014-06-28,1,1.0
282047,2013-10-10,NCELGW2014-06-28,1,1.0


In [130]:
search_created_zip[search_created_zip['zip'] == 'BOGTUL2013-01-15']

,created_date,zip,fake_columns,rank_search
7209,2013-01-08,BOGTUL2013-01-15,1,1.0
388368,2013-01-08,BOGTUL2013-01-15,1,2.0


In [131]:
# Deleting fake column created as it is not needed anymore:
search_created_zip.drop('fake_columns', axis = 1, inplace = True)

In [132]:
search_created_zip

,created_date,zip,rank_search
0,2013-01-01,TXLAUH2013-01-26,1.0
1,2013-01-01,ATHMIL2013-01-04,1.0
2,2013-01-01,ICTSFO2013-08-02,1.0
3,2013-01-01,RNBARN2013-01-02,1.0
4,2013-01-01,OSLMAD2013-03-22,1.0
...,...,...,...
388364,2013-01-08,HLZBNE2013-10-26,2.0
388365,2013-01-08,PEKSEL2013-03-22,2.0
388366,2013-01-08,YULNCL2013-01-27,2.0
388367,2013-01-08,MIANCE2013-01-23,2.0


Same steps for the bookings

In [133]:
bookings_datetime

,created_date,board_date,Origin,Destination
1,2013-03-26,2013-04-12,SAL,CLT
2,2013-03-26,2013-07-15,SAL,CLT
3,2013-03-26,2013-04-24,AKL,SVO
4,2013-03-26,2013-05-14,AKL,SVO
5,2013-03-20,2013-04-07,DEN,LGA
...,...,...,...,...
999994,2013-05-14,2013-05-24,CMB,MAA
999996,2013-05-18,2013-05-19,CMB,MAA
999997,2013-05-28,2013-07-12,AUS,RDU
999998,2013-05-28,2013-06-04,TLS,ORY


In [134]:
bookings_rank = bookings_datetime.copy()

#Converting board date to str as later am going to zip it and create a string from a tuple

bookings_rank['board_date'] = bookings_rank['board_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

#Zipping columns to obtaing an Id name with Origin, Destination and board date 

bookings_rank['zip'] = tuple(zip(bookings_rank.Origin, bookings_rank.Destination, bookings_rank.board_date))

#Droping columns we dont need anymore

bookings_created_zip = bookings_rank.drop(['Origin', 'Destination', 'board_date'], axis = 1)

#Converting tuple to string 

bookings_created_zip['zip'] = bookings_created_zip ['zip'].apply(lambda x: ''.join(x))

#I make sure created-date its in datetime format as its going to be needed later for the groupby

bookings_created_zip ['created_date'] =  pd.to_datetime(bookings_created_zip['created_date'])

# I cant groupby by created_date and zip without a value to groupby with.
# Becouse of this a fake columns is created with a value of 1 to be able to groupby by created date and zip
bookings_created_zip ['fake_columns'] = 1

# Groupby

bookings_created_zip['rank_book'] = bookings_created_zip.groupby(['created_date', 'zip']).rank(method= 'first')




In [135]:
#Drop the fake column we created to bea able to do the groupby
bookings_created_zip.drop('fake_columns', axis = 1, inplace= True)

In [141]:
bookings_created_zip

,created_date,zip,rank_book
1,2013-03-26,SALCLT2013-04-12,1.0
2,2013-03-26,SALCLT2013-07-15,1.0
3,2013-03-26,AKLSVO2013-04-24,1.0
4,2013-03-26,AKLSVO2013-05-14,1.0
5,2013-03-20,DENLGA2013-04-07,1.0
...,...,...,...
999994,2013-05-14,CMBMAA2013-05-24,1.0
999996,2013-05-18,CMBMAA2013-05-19,1.0
999997,2013-05-28,AUSRDU2013-07-12,1.0
999998,2013-05-28,TLSORY2013-06-04,1.0


In [142]:
search_created_zip

,created_date,zip,rank_search
0,2013-01-01,TXLAUH2013-01-26,1.0
1,2013-01-01,ATHMIL2013-01-04,1.0
2,2013-01-01,ICTSFO2013-08-02,1.0
3,2013-01-01,RNBARN2013-01-02,1.0
4,2013-01-01,OSLMAD2013-03-22,1.0
...,...,...,...
388364,2013-01-08,HLZBNE2013-10-26,2.0
388365,2013-01-08,PEKSEL2013-03-22,2.0
388366,2013-01-08,YULNCL2013-01-27,2.0
388367,2013-01-08,MIANCE2013-01-23,2.0


# Merge

In [137]:
merg = search_created_zip.merge(bookings_created_zip, how='left')

In [138]:
merg

,created_date,zip,rank_search,rank_book
0,2013-01-01,TXLAUH2013-01-26,1.0,NaN
1,2013-01-01,ATHMIL2013-01-04,1.0,NaN
2,2013-01-01,ICTSFO2013-08-02,1.0,NaN
3,2013-01-01,RNBARN2013-01-02,1.0,NaN
4,2013-01-01,OSLMAD2013-03-22,1.0,NaN
...,...,...,...,...
387321,2013-01-08,HLZBNE2013-10-26,2.0,NaN
387322,2013-01-08,PEKSEL2013-03-22,2.0,NaN
387323,2013-01-08,YULNCL2013-01-27,2.0,NaN
387324,2013-01-08,MIANCE2013-01-23,2.0,NaN


In [140]:
merg[(merg['rank_book'] == 1.0) & (merg['rank_search'] == 1.0)]

,created_date,zip,rank_search,rank_book
430,2013-01-01,RUHJED2013-01-14,1.0,1.0
593,2013-01-01,DMEBKK2013-01-29,1.0,1.0
737,2013-01-01,JEDRUH2013-01-04,1.0,1.0
919,2013-01-01,DELBOM2013-01-02,1.0,1.0
1146,2013-01-02,LHRSYD2013-04-22,1.0,1.0
...,...,...,...,...
347759,2013-12-15,CAIDXB2013-12-17,1.0,1.0
350077,2013-12-18,DRSFCO2014-10-16,1.0,1.0
350341,2013-12-18,SFOTPE2014-02-08,1.0,1.0
350702,2013-12-18,DENYYZ2014-01-06,1.0,1.0
